In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DS203_Final_Project").getOrCreate()

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/11 20:01:32 WARN Utils: Your hostname, Carolinas-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.115 instead (on interface en0)
25/12/11 20:01:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/11 20:01:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.csv("data/Chicago_Crim*.csv", header=True, inferSchema=True)
df.show(5)

25/12/11 20:01:38 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: data/Chicago_Crim*.csv.
java.io.FileNotFoundException: File data/Chicago_Crim*.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.analysis.Resolve

+----+-------+-----------+--------------------+-------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
| _c0|     ID|Case Number|                Date|              Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+----+-------+-----------+--------------------+-------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
| 879|4786321|   HM399414|01/01/2004 12:01:...|  082XX S COLES AVE|0840|               THEFT|FINANCIAL ID THEF...| 

25/12/11 20:01:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, Beat, District, Ward, Community Area, FBI Code, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Location
 Schema: _c0, ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, Beat, District, Ward, Community Area, FBI Code, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Location
Expected: _c0 but found: 
CSV file: file:///Users/carolinagarza/DS203_Final_Spark/data/Chicago_Crimes_2001_to_2004.csv


In [3]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [4]:
num_rows = df.count()
num_cols = len(df.columns)
num_rows, num_cols

(7941286, 23)

In [5]:
df.describe().show()

25/12/11 20:03:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/12/11 20:03:12 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, Beat, District, Ward, Community Area, FBI Code, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Location
 Schema: _c0, ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, Beat, District, Ward, Community Area, FBI Code, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Location
Expected: _c0 but found: 
CSV file: file:///Users/carolinagarza/DS203_Final_Spark/data/Chicago_Crimes_2001_to_2004.csv
25/12/11 20:03:12 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , ID, Case Number, Date, Block, IUCR

+-------+------------------+------------------+------------------+--------------------+------------------+-----------------+-----------------+---------------+--------------------+-------+--------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-------------+--------------------+-------------------+--------------------+
|summary|               _c0|                ID|       Case Number|                Date|             Block|             IUCR|     Primary Type|    Description|Location Description| Arrest|Domestic|              Beat|          District|              Ward|    Community Area|          FBI Code|      X Coordinate|      Y Coordinate|             Year|   Updated On|            Latitude|          Longitude|            Location|
+-------+------------------+------------------+------------------+--------------------+------------------+-----------------+-----------------+----------

In [6]:
df.filter(df["Primary Type"] == "THEFT").show(5)

+----+-------+-----------+--------------------+-------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
| _c0|     ID|Case Number|                Date|              Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+----+-------+-----------+--------------------+-------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
| 879|4786321|   HM399414|01/01/2004 12:01:...|  082XX S COLES AVE|0840|       THEFT|FINANCIAL ID THEF...|           RESIDENCE| False|   Fa

25/12/11 20:06:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, Beat, District, Ward, Community Area, FBI Code, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Location
 Schema: _c0, ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, Beat, District, Ward, Community Area, FBI Code, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Location
Expected: _c0 but found: 
CSV file: file:///Users/carolinagarza/DS203_Final_Spark/data/Chicago_Crimes_2001_to_2004.csv


In [7]:
df.filter(df["Arrest"] == "True").show(5)

+-----+-------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|  _c0|     ID|Case Number|                Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+-----+-------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
| 3756|4791194|   HM403711|01/01/2001 11:00:...|114XX S ST LAWREN...|0266| CRIM SEXUAL ASSAULT|           PRE

25/12/11 20:06:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, Beat, District, Ward, Community Area, FBI Code, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Location
 Schema: _c0, ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, Beat, District, Ward, Community Area, FBI Code, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Location
Expected: _c0 but found: 
CSV file: file:///Users/carolinagarza/DS203_Final_Spark/data/Chicago_Crimes_2001_to_2004.csv


In [8]:
df.filter((df["Domestic"] == "True") & (df["Arrest"] == "True")).show(5)

+-----+-------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|  _c0|     ID|Case Number|                Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+-----+-------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
| 3756|4791194|   HM403711|01/01/2001 11:00:...|114XX S ST LAWREN...|0266| CRIM SEXUAL ASSAULT|           PRE

25/12/11 20:07:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, Beat, District, Ward, Community Area, FBI Code, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Location
 Schema: _c0, ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, Beat, District, Ward, Community Area, FBI Code, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Location
Expected: _c0 but found: 
CSV file: file:///Users/carolinagarza/DS203_Final_Spark/data/Chicago_Crimes_2001_to_2004.csv


In [9]:
df.filter((df["Primary Type"] == "BATTERY") | (df["Primary Type"] == "ASSAULT")).show(5)

+------+-------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|   _c0|     ID|Case Number|                Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+------+-------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
| 30374|4834746|   HM440439|06/27/2004 03:30:...|106XX S MICHIGAN AVE|0460|     BATTERY|              SIMPLE|  SMALL RETAIL STORE| False|   False| 512|     5.0| 9.0|          49.0|     08B| 

25/12/11 20:07:35 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, Beat, District, Ward, Community Area, FBI Code, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Location
 Schema: _c0, ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, Beat, District, Ward, Community Area, FBI Code, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Location
Expected: _c0 but found: 
CSV file: file:///Users/carolinagarza/DS203_Final_Spark/data/Chicago_Crimes_2001_to_2004.csv


In [10]:
df.filter(df["Year"] == "2016").show(5)

25/12/11 20:08:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, Beat, District, Ward, Community Area, FBI Code, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Location
 Schema: _c0, ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, Beat, District, Ward, Community Area, FBI Code, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Location
Expected: _c0 but found: 
CSV file: file:///Users/carolinagarza/DS203_Final_Spark/data/Chicago_Crimes_2001_to_2004.csv
25/12/11 20:08:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , ID, Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Arrest, Domestic, Beat, District, Ward, Community Area, FBI Code, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Locat

+---+--------+-----------+--------------------+-------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|_c0|      ID|Case Number|                Date|              Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+---+--------+-----------+--------------------+-------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|  3|10508693|   HZ250496|05/03/2016 11:40:...| 013XX S SAWYER AVE|0486|             BATTERY|DOMESTIC BATTERY ...| 

In [11]:
from pyspark.sql.functions import col

df = df.drop("_c0")
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [13]:
from pyspark.sql.functions import when, col

df2 = df.withColumn(
    "Arrest_Flag",
    when(col("Arrest") == "True", 1)   # si Arrest == "True" → 1
    .when(col("Arrest") == "False", 0) # si Arrest == "False" → 0
    .otherwise(None)                   # por si hay valores raros
)

df2.select("Arrest", "Arrest_Flag").show(10)

+------+-----------+
|Arrest|Arrest_Flag|
+------+-----------+
| False|          0|
| False|          0|
| False|          0|
| False|          0|
| False|          0|
| False|          0|
|  True|          1|
| False|          0|
| False|          0|
| False|          0|
+------+-----------+
only showing top 10 rows


In [14]:
df2.groupBy("Arrest_Flag").count().show()

+-----------+-------+
|Arrest_Flag|  count|
+-----------+-------+
|          1|2249420|
|          0|5691865|
|       NULL|      1|
+-----------+-------+



In [15]:
from pyspark.sql.functions import round

total = df2.count()

df2.groupBy("Arrest_Flag") \
   .count() \
   .withColumn("percentage", round(col("count") / total * 100, 2)) \
   .show()

+-----------+-------+----------+
|Arrest_Flag|  count|percentage|
+-----------+-------+----------+
|          1|2249420|     28.33|
|          0|5691865|     71.67|
|       NULL|      1|       0.0|
+-----------+-------+----------+



In [16]:
df2.groupBy("Primary Type", "Arrest_Flag").count().show(20)

+-------------------+-----------+-------+
|       Primary Type|Arrest_Flag|  count|
+-------------------+-----------+-------+
|  CRIMINAL TRESPASS|          1| 168135|
|           GAMBLING|          1|  18632|
|CRIM SEXUAL ASSAULT|          0|  24931|
|  CRIMINAL TRESPASS|          0|  61232|
|              THEFT|          0|1444055|
|              ARSON|          0|  11371|
|   PUBLIC INDECENCY|          1|    161|
|        SEX OFFENSE|          1|   9168|
|           HOMICIDE|          1|   4501|
|       INTIMIDATION|          0|   3801|
|              ARSON|          1|   1726|
|  WEAPONS VIOLATION|          1|  62044|
|           STALKING|          0|   3127|
|MOTOR VEHICLE THEFT|          1|  33209|
|  DOMESTIC VIOLENCE|          1|      2|
|         KIDNAPPING|          1|    855|
|    CRIMINAL DAMAGE|          0| 857611|
|            BATTERY|          0|1112665|
|          NARCOTICS|          1| 878459|
|            ROBBERY|          0| 270736|
+-------------------+-----------+-

In [17]:
df_arrest = df2.filter(col("Arrest_Flag") == 1)
df_arrest.show(5)

+-------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-----------+
|     ID|Case Number|                Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|Arrest_Flag|
+-------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-----------+
|4791194|   HM403711|01/01/2001 11:00:...|114XX S ST LAWREN...|0266| CRIM SEXUAL ASSAULT|  

In [19]:
df2 = df2.withColumn("Year", col("Year").cast("int"))

In [20]:
df2 = df2.withColumn("X Coordinate", col("X Coordinate").cast("double"))

In [23]:
from pyspark.sql.functions import count, countDistinct

# Aggregate 1: total de registros
df2.select(count("*").alias("total_rows")).show()

# Aggregate 2: número de tipos de crimen distintos
df2.select(countDistinct("Primary Type").alias("num_primary_types")).show()

+----------+
|total_rows|
+----------+
|   7941286|
+----------+



+-----------------+
|num_primary_types|
+-----------------+
|               36|
+-----------------+



In [24]:
df2.select(count("*").alias("total_rows")).show()
df2.select(countDistinct("Primary Type").alias("num_primary_types")).show()

+----------+
|total_rows|
+----------+
|   7941286|
+----------+



+-----------------+
|num_primary_types|
+-----------------+
|               36|
+-----------------+



In [26]:
from pyspark.sql.functions import count, countDistinct


df2.select(
    count("*").alias("total_rows")
).show()


df2.select(
    countDistinct("Primary Type").alias("num_primary_types")
).show()

+----------+
|total_rows|
+----------+
|   7941286|
+----------+



+-----------------+
|num_primary_types|
+-----------------+
|               36|
+-----------------+



In [27]:
from pyspark.sql.functions import count


df2.groupBy("Primary Type") \
   .agg(count("*").alias("count_crimes")) \
   .orderBy(col("count_crimes").desc()) \
   .show(10)

+-------------------+------------+
|       Primary Type|count_crimes|
+-------------------+------------+
|              THEFT|     1640506|
|            BATTERY|     1442717|
|    CRIMINAL DAMAGE|      923000|
|          NARCOTICS|      885431|
|      OTHER OFFENSE|      491923|
|            ASSAULT|      481661|
|           BURGLARY|      470958|
|MOTOR VEHICLE THEFT|      370548|
|            ROBBERY|      300453|
| DECEPTIVE PRACTICE|      280931|
+-------------------+------------+
only showing top 10 rows


In [29]:
from pyspark.sql.functions import count, col

crimes_per_year = df.groupBy("Year") \
    .agg(count("*").alias("crimes_per_year")) \
    .orderBy("Year")

crimes_per_year.show()

+------------+---------------+
|        Year|crimes_per_year|
+------------+---------------+
|        2001|         568517|
|        2002|         490879|
|        2003|         475913|
|        2004|         388205|
|        2005|         455811|
|        2006|         794685|
|        2007|         621849|
|        2008|         852053|
|        2009|         783900|
|        2010|         700692|
|        2011|         352066|
|        2012|         335670|
|        2013|         306703|
|        2014|         274527|
|        2015|         262995|
|        2016|         265462|
|        2017|          11357|
|41.789832136|              1|
|Y Coordinate|              1|
+------------+---------------+



In [30]:
df2.groupBy("Primary Type") \
   .agg(count("*").alias("count_crimes")) \
   .orderBy(col("count_crimes").asc()) \
   .show(10)

+--------------------+------------+
|        Primary Type|count_crimes|
+--------------------+------------+
|                IUCR|           1|
|   DOMESTIC VIOLENCE|           2|
|NON-CRIMINAL (SUB...|           4|
|   HUMAN TRAFFICKING|          28|
|           RITUALISM|          31|
|      NON - CRIMINAL|          38|
|CONCEALED CARRY L...|          90|
|        NON-CRIMINAL|          97|
|OTHER NARCOTIC VI...|         144|
|    PUBLIC INDECENCY|         163|
+--------------------+------------+
only showing top 10 rows


In [31]:
df.groupBy("Year") \
   .agg(count("*").alias("crimes_per_year")) \
   .orderBy(col("crimes_per_year").desc()) \
   .show()

+------------+---------------+
|        Year|crimes_per_year|
+------------+---------------+
|        2008|         852053|
|        2006|         794685|
|        2009|         783900|
|        2010|         700692|
|        2007|         621849|
|        2001|         568517|
|        2002|         490879|
|        2003|         475913|
|        2005|         455811|
|        2004|         388205|
|        2011|         352066|
|        2012|         335670|
|        2013|         306703|
|        2014|         274527|
|        2016|         265462|
|        2015|         262995|
|        2017|          11357|
|Y Coordinate|              1|
|41.789832136|              1|
+------------+---------------+



In [32]:

crime_totals = df2.groupBy("Primary Type") \
    .agg(count("*").alias("total_by_type"))


joined_df = df2.join(crime_totals, on="Primary Type", how="left")

joined_df.show(5)

+--------------------+-------+-----------+--------------------+-------------------+----+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-----------+-------------+
|        Primary Type|     ID|Case Number|                Date|              Block|IUCR|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|Arrest_Flag|total_by_type|
+--------------------+-------+-----------+--------------------+-------------------+----+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-----------+-------------+
|OFFENSE INVOLVING...|4789749|   HM402220|06/20/2004

In [35]:
from pyspark.sql.functions import count, col, rank, lag, lead
from pyspark.sql.window import Window


df2 = df.drop("_c0")


crime_totals = df2.groupBy("Primary Type") \
    .agg(count("*").alias("total_by_type"))

crime_totals.show(5)


w = Window.orderBy(col("total_by_type").desc())


ranked_df = crime_totals.withColumn("rank", rank().over(w))
ranked_df.show(10)


ranked_df2 = ranked_df.withColumn("prev_total", lag("total_by_type").over(w)) \
                      .withColumn("next_total", lead("total_by_type").over(w))

ranked_df2.show(10)

25/12/11 20:55:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/11 20:55:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/11 20:55:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+-------------+
|        Primary Type|total_by_type|
+--------------------+-------------+
|OFFENSE INVOLVING...|        51441|
|            STALKING|         3734|
|PUBLIC PEACE VIOL...|        58548|
|           OBSCENITY|          496|
|               ARSON|        13097|
+--------------------+-------------+
only showing top 5 rows


25/12/11 20:55:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/11 20:55:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/11 20:55:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/11 20:55:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/11 20:55:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/11 20:55:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------------+-------------+----+
|       Primary Type|total_by_type|rank|
+-------------------+-------------+----+
|              THEFT|      1640506|   1|
|            BATTERY|      1442717|   2|
|    CRIMINAL DAMAGE|       923000|   3|
|          NARCOTICS|       885431|   4|
|      OTHER OFFENSE|       491923|   5|
|            ASSAULT|       481661|   6|
|           BURGLARY|       470958|   7|
|MOTOR VEHICLE THEFT|       370548|   8|
|            ROBBERY|       300453|   9|
| DECEPTIVE PRACTICE|       280931|  10|
+-------------------+-------------+----+
only showing top 10 rows


25/12/11 20:55:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/11 20:55:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/11 20:55:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/11 20:55:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------------+-------------+----+----------+----------+
|       Primary Type|total_by_type|rank|prev_total|next_total|
+-------------------+-------------+----+----------+----------+
|              THEFT|      1640506|   1|      NULL|   1442717|
|            BATTERY|      1442717|   2|   1640506|    923000|
|    CRIMINAL DAMAGE|       923000|   3|   1442717|    885431|
|          NARCOTICS|       885431|   4|    923000|    491923|
|      OTHER OFFENSE|       491923|   5|    885431|    481661|
|            ASSAULT|       481661|   6|    491923|    470958|
|           BURGLARY|       470958|   7|    481661|    370548|
|MOTOR VEHICLE THEFT|       370548|   8|    470958|    300453|
|            ROBBERY|       300453|   9|    370548|    280931|
| DECEPTIVE PRACTICE|       280931|  10|    300453|    229367|
+-------------------+-------------+----+----------+----------+
only showing top 10 rows


25/12/11 21:10:29 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 124412 ms exceeds timeout 120000 ms
25/12/11 21:10:29 WARN SparkContext: Killing executors is not supported by current scheduler.
25/12/11 21:10:29 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:669)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1296)
	at o